In [7]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf

/Users/mark/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Annotation

FPN (pre-trained) identifes key points and labels them

In [ ]:
from PIL import Image
import numpy as np

def load_image( path ) :
    img = Image.open( path )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

# sample_image_path = "/kaggle/input/geometry-3k/train/train/0/img_diagram.png"
# sample_image = load_image(sample_image_path)

In [ ]:
# sample_image.shape

(434, 864, 4)

## ResNet

I'm making a implenation myself based on the orgional paper (34 layer version) to test my knowldege. The 50 layer version with alredy fitted weights will be use for the bottom to top part of the FPN.

In [48]:
import keras

class ResNetBlock(keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=1):
        super(ResNetBlock, self).__init__()
        self.dotted = (strides != 1)

        self.conv1 = keras.layers.Conv2D(filters, kernel_size, strides, padding="same")
        self.bn1 = keras.layers.BatchNormalization()
        self.conv2 = keras.layers.Conv2D(filters, kernel_size, padding="same")
        self.bn2 = keras.layers.BatchNormalization()
        self.activation = keras.layers.ReLU()

        if self.dotted: ## fix dimentions (option b in paper)
            self.bn3 = keras.layers.BatchNormalization()
            self.conv3 = keras.layers.Conv2D(filters, 1, strides)

    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)

        shortcut = inputs
        if self.dotted:
            shortcut = self.bn3(shortcut)
            shortcut = self.conv3(shortcut)
        
        x = keras.layers.add([x, shortcut]) ##skip connections
        x = self.activation(x)

        return x
    
class ResNet(keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()

        self.conv1 = keras.layers.Conv2D(64, 7, 2, padding='same')
        self.bn = keras.layers.BatchNormalization()
        self.relu = keras.layers.ReLU()
        self.mpl = keras.layers.MaxPooling2D(3, 2)

        self.block1 = ResNetBlock(64, 3)
        self.block2 = ResNetBlock(64, 3)
        self.block3 = ResNetBlock(64, 3)

        self.block4 = ResNetBlock(128, 3, 2)
        self.block5 = ResNetBlock(128, 3)
        self.block6 = ResNetBlock(128, 3)
        self.block7 = ResNetBlock(128, 3)

        self.block8 = ResNetBlock(256, 3, 2)
        self.block9 = ResNetBlock(256, 3)
        self.block10 = ResNetBlock(256, 3)
        self.block11 = ResNetBlock(256, 3)
        self.block12 = ResNetBlock(256, 3)
        self.block13 = ResNetBlock(256, 3)

        self.block14 = ResNetBlock(512, 3, 2)
        self.block15 = ResNetBlock(512, 3)
        self.block16 = ResNetBlock(512, 3)

        self.pool = keras.layers.GlobalAveragePooling2D()
        self.fc1 = keras.layers.Dense(512, activation='relu')
        self.dp1 = keras.layers.Dropout(0.5)
        self.fc2 = keras.layers.Dense(512, activation='relu')
        self.dp2 = keras.layers.Dropout(0.5)
        self.fc3 = keras.layers.Dense(64)

    def call(self, x):
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.mpl(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)

        x = self.block4(x)
        x = self.block5(x)
        x = self.block6(x)
        x = self.block7(x)

        x = self.block8(x)
        x = self.block9(x)
        x = self.block10(x)
        x = self.block11(x)
        x = self.block12(x)
        x = self.block13(x)

        x = self.block14(x)
        x = self.block15(x)
        x = self.block16(x)

        x = self.pool(x)
        x = self.fc1(x)
        x = self.dp1(x)
        x = self.fc2(x)
        x = self.dp2(x)
        x = self.fc3(x)
        
        return x    

In [50]:
model = ResNet()

input_test = tf.random.normal([1, 480, 480, 3])
model(input_test)
model.summary()

Model: "res_net_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_304 (Conv2D)             │ (1, 240, 240, 64)      │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_297         │ (1, 240, 240, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_141 (ReLU)                │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_132 (ResNetBlock) │ ?                      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_133 (ResNetBlock) │ ?                      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_134 (ResNetBlock) │ ?                      │        74,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_135 (ResNetBlock) │ ?                      │       231,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_136 (ResNetBlock) │ ?                      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_137 (ResNetBlock) │ ?                      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_138 (ResNetBlock) │ ?                      │       296,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_139 (ResNetBlock) │ ?                      │       920,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_140 (ResNetBlock) │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_141 (ResNetBlock) │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_142 (ResNetBlock) │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_143 (ResNetBlock) │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_144 (ResNetBlock) │ ?                      │     1,182,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_145 (ResNetBlock) │ ?                      │     3,676,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_146 (ResNetBlock) │ ?                      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net_block_147 (ResNetBlock) │ ?                      │     4,723,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_8      │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (1, 512)               │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (1, 512)               │       262,65

 Total params: 21,866,560 (83.41 MB)

 Trainable params: 21,850,432 (83.35 MB)

 Non-trainable params: 16,128 (63.00 KB)